In [1]:
import recordlinkage
from Annotation_helper import *
import numpy as np

In [ ]:
#Set all settings for the resulting figures

import seaborn as sns
sns.set()

SMALL_SIZE = 15
MEDIUM_SIZE = 17
BIGGER_SIZE = 19

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.tight_layout()

In [4]:
data_folder = "../../Data"
structured_data_folder = data_folder + "/structured_data"
text_path = data_folder + "/text"
data_path = structured_data_folder + "/test.csv"
dist_folder = structured_data_folder + "/Distances/"
figure_folder = "../Figures/Results/Embeddings"
model_folder = "../Models/Embeddings"
matches = pd.read_csv(structured_data_folder + "/Matches/Bas3.csv", index_col=0)
figure_folder = "../Figures/Distances"

In [5]:
transactions = load_transactions("../../Data/structured_data\VOC Cochin Slave Transactions 1706-1801 - IISH - Database - 2018 - v1.csv")
permissions = load_permissions("../../Data/structured_data\VOC Cochin Slave Transport Permissions 1770-1795 - IISH - Database - 2018 - v1.csv")

# Generate distances between datasets
(Can take long)

In [11]:
dist_metric = 'levenshtein'
indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(permissions, transactions)

compare_cl = recordlinkage.Compare()

compare_cl.exact('SlaafGender', 'SlaafGender', label='SlaafGender')
compare_cl.string('SlaafNaamNieuw', 'SlaafNaamNieuw', label='SlaafNaamNieuw', method=dist_metric)
compare_cl.exact('BezitterGender', 'KoperGender', label='BezitterGender')
compare_cl.string('BezitterVoornaam', 'KoperVoornaam', label='BezitterVoornaam', method=dist_metric)
compare_cl.string('BezitterTussen', 'KoperTussen', label='BezitterTussen', method=dist_metric)
compare_cl.string('BezitterAchternaam', 'KoperAchternaam', label='BezitterAchternaam', method=dist_metric)
compare_cl.string('BezitterBeroep', 'KoperBeroep', label='BezitterBeroep', method=dist_metric)
compare_cl.string('BezitterStatus', 'KoperStatus', label='BezitterStatus', method=dist_metric)
compare_cl.string('BezitterEtniciteit', 'KoperEtniciteit', label='BezitterEtniciteit', method=dist_metric)
n_batches = 100
split_perm = np.array_split(permissions, n_batches)

Save the resulting distances to multiple (100) files

In [12]:
for i, p in enumerate(split_perm):
    print(f"[{i}/{n_batches}]")
    candidate_links = indexer.index(p, transactions)
    features = compare_cl.compute(candidate_links, p, transactions)
    features = features.reset_index()
    features = features.rename(columns={"level_0": "permission_indx", "level_1": "transaction_indx"})
    features.to_csv(f"{dist_folder}distances_{i}.csv", index=False)
    clear_output(wait=True)

[99/100]


# Load distances for testing

In [13]:
distances = pd.read_csv(dist_folder + "/distances_0.csv", dtype={'permission_indx': 'uint16', 'transaction_indx': 'uint16', 'SlaafGender': 'bool', 'SlaafNaamNieuw': 'float16', 'BezitterGender': 'bool', 'BezitterVoornaam': 'float16', 'BezitterBeroep': 'float16', 'BezitterAchternaam': 'float16'})
for i in range(1, 99):
    df2 = pd.read_csv(f"{dist_folder}/distances_{i}.csv", dtype={'permission_indx': 'uint16', 'transaction_indx': 'uint16', 'SlaafGender': 'bool', 'SlaafNaamNieuw': 'float16', 'BezitterGender': 'bool', 'BezitterVoornaam': 'float16', 'BezitterBeroep': 'float16', 'BezitterAchternaam': 'float16'})
    distances = pd.concat([distances, df2])
distances.reset_index(inplace=True)

In [14]:
distances

,index,permission_indx,transaction_indx,SlaafGender,SlaafNaamNieuw,BezitterGender,BezitterVoornaam,BezitterTussen,BezitterAchternaam,BezitterBeroep,BezitterStatus,BezitterEtniciteit
0,0,0,0,True,0.000000,False,0.000000,0.0,0.000000,0.0,0.0,0.0
1,1,0,1,True,0.250000,False,0.000000,0.0,0.000000,0.0,0.0,0.0
2,2,0,2,False,0.166626,False,0.000000,0.0,0.000000,0.0,0.0,0.0
3,3,0,3,True,0.111084,False,0.000000,0.0,0.000000,0.0,0.0,0.0
4,4,0,4,True,0.142822,False,0.000000,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
27314395,273139,3888,7192,True,0.142822,True,0.166626,0.0,0.166626,0.0,0.0,1.0
27314396,273140,3888,7193,False,0.142822,True,0.166626,0.0,0.166626,0.0,0.0,1.0
27314397,273141,3888,7194,False,0.000000,True,0.277832,0.0,0.166626,0.0,0.0,1.0
27314398,273142,3888,7195,False,0.111084,True,0.277832,0.0,0.166626,0.0,0.0,1.0
